### Exercise 1

In [3]:
import pickle 
from os.path import exists

if not exists(r'retail-raw.pkl'):
    import requests

    # URL of the data
    url = "https://goo.gl/wi8KHg"

    # Fetch the content from the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        retail_raw = response.text.splitlines()
        # Now, retail_raw is equivalent to the content returned by readLines() in R
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
else:
    with open("retail-raw.pkl", 'rb') as f:
        retail_raw = pickle.load(f)

In [4]:
with open("retail-raw.pkl", 'rb') as f:
    retail_raw = pickle.load(f)

In [5]:
import pandas as pd

retail_margin = pd.read_csv("retail-margin.csv").iloc[:, 1:]

In [6]:
retail_margin.head()

,margin,item,price
0,24.231,item1,498.99
1,4.447,item10,54.99
2,0.431,item100,2.19
3,-0.534,item101,0.49
4,1.593,item102,13.29


In [7]:
import numpy as np

retail_list_ = [len(x) for x in retail_raw]
print(f'median length: {np.median(retail_list_)}')
print(f"smallest lenght: {np.min(retail_list_)}")
print(f'largest length: {np.max(retail_list_)}')

import gc
del retail_list_
gc.collect()

median length: 109.0
smallest lenght: 5
largest length: 324


0

In [31]:
retail_margin.shape

(830, 3)

In [8]:
retail_trans = pd.DataFrame(0, index=range(len(retail_raw)), columns=range(retail_margin.shape[0]))
retail_trans.columns = retail_margin["item"]

In [12]:
from tqdm import tqdm
for row, trans in enumerate(tqdm(retail_raw)):
    for item in trans.split(","):
        try:
            retail_trans.loc[row, item] = 1
        except:
            print(row, item)

In [13]:
retail_trans.head()

item,item1,item10,item100,item101,item102,item103,item104,item105,item106,item107,...,item9,item90,item91,item92,item93,item94,item95,item96,item97,item98
0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

retail_trans = retail_trans.astype(bool)
# Assuming 'retail_trans' is a pandas DataFrame containing the transaction data in a one-hot encoded format
# (where columns are items and rows are transactions)

# Apply the apriori algorithm
frequent_itemsets = apriori(retail_trans, min_support=0.02, use_colnames=True, max_len=3)

# Generate association rules with a minimum confidence of 0.2
retail_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

# Display the resulting rules
print(retail_rules)